# Importing the modules

In [3]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.layers.experimental import preprocessing
import time
import pandas as pd
from shutil import copy2

In [1]:
train_directory = 'tiny-imagenet-200/tiny-imagenet-200/train/'
word_files = 'tiny-imagenet-200/tiny-imagenet-200/words.txt'

In [2]:
dst = 'tiny-imagenet-200/tiny-imagenet-200/train_all/'

In [20]:
k=0 
for cat in os.listdir(train_directory):
    
    image_directory = train_directory + cat + '/images/'
    for image in os.listdir(image_directory):
        copy2(image_directory + image, dst)
    print(k)
    k+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


# Checking Categories

In [4]:
dictionary = {}
k= 0
for i in os.listdir(train_directory):
    dictionary[i] = k
    k += 1
print(dictionary)

{'n01443537': 0, 'n01629819': 1, 'n01641577': 2, 'n01644900': 3, 'n01698640': 4, 'n01742172': 5, 'n01768244': 6, 'n01770393': 7, 'n01774384': 8, 'n01774750': 9, 'n01784675': 10, 'n01855672': 11, 'n01882714': 12, 'n01910747': 13, 'n01917289': 14, 'n01944390': 15, 'n01945685': 16, 'n01950731': 17, 'n01983481': 18, 'n01984695': 19, 'n02002724': 20, 'n02056570': 21, 'n02058221': 22, 'n02074367': 23, 'n02085620': 24, 'n02094433': 25, 'n02099601': 26, 'n02099712': 27, 'n02106662': 28, 'n02113799': 29, 'n02123045': 30, 'n02123394': 31, 'n02124075': 32, 'n02125311': 33, 'n02129165': 34, 'n02132136': 35, 'n02165456': 36, 'n02190166': 37, 'n02206856': 38, 'n02226429': 39, 'n02231487': 40, 'n02233338': 41, 'n02236044': 42, 'n02268443': 43, 'n02279972': 44, 'n02281406': 45, 'n02321529': 46, 'n02364673': 47, 'n02395406': 48, 'n02403003': 49, 'n02410509': 50, 'n02415577': 51, 'n02423022': 52, 'n02437312': 53, 'n02480495': 54, 'n02481823': 55, 'n02486410': 56, 'n02504458': 57, 'n02509815': 58, 'n0266

In [10]:
train_images = 'tiny-imagenet-200/tiny-imagenet-200/train_all/'

In [6]:
def one_hot(input_value, length):
    output = [0]* length
    output[input_value] = 1
    return output
one_hot(8,20)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
x = np.zeros([100000, 64, 64, 3])
y = np.zeros([100000, 200])

for i,j in enumerate(os.listdir(train_images)):
    image = tf.keras.preprocessing.image.load_img(train_images + j, color_mode='rgb')

    x[i] = (tf.keras.preprocessing.image.img_to_array(image))
    y[i] = (one_hot(dictionary[j[0:9]], 200))
                
    if(i% 1000 == 0):
        print(i)

In [14]:
with tf.device('CPU:0'):
    x = tf.Variable(tf.zeros([100, 64, 64, 3]))
    y = tf.Variable(tf.zeros([100, 200]))

    for i,j in enumerate(os.listdir(train_images)):
        image = tf.keras.preprocessing.image.load_img(train_images + j, color_mode='rgb')

        x[i].assign(tf.keras.preprocessing.image.img_to_array(image))
        y[i].assign(tf.one_hot(tf.constant(dictionary[j[0:9]], 200)))
                    
        if(i >= 99):
            break

TypeError: Cannot convert the argument `type_value`: 200 to a TensorFlow DType.

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices({"x": x[0:80], "y": y[0:80]})
val_dataset = tf.data.Dataset.from_tensor_slices({"x": x[80:], "y": y[80:]})
